# 深度学习核心：数值稳定性、模型初始化与激活函数

## 1. 数值稳定性：梯度消失与梯度爆炸

在深度神经网络中，前向传播是从输入层一层层算到输出层，而反向传播（更新参数）是从输出层一层层往回算。问题就出在这个“往回算”的连乘机制上。

假设我们有一个 $d$ 层的简单多层感知机（MLP），为了简化，我们暂时不看偏置和激活函数，每一层的计算可以抽象为矩阵乘法：
$$h^{(l)} = W^{(l)} h^{(l-1)}$$



当我们计算关于第 $l$ 层权重的梯度时，根据链式法则 (Chain Rule)，梯度是由后面所有层的导数连乘得到的：
$$\frac{\partial L}{\partial W^{(l)}} = \frac{\partial L}{\partial h^{(d)}} \left( \prod_{i=l}^{d-1} \frac{\partial h^{(i+1)}}{\partial h^{(i)}} \right) \frac{\partial h^{(l)}}{\partial W^{(l)}}$$

这里的核心是中间那个连乘项 $\prod_{i=l}^{d-1} W^{(i+1)}$。想象一下，如果有 100 层网络：
* **梯度消失 (Gradient Vanishing)**：如果矩阵 $W$ 里面的值大多小于 1（比如 0.9），那么 $0.9^{100} \approx 0.00002$。梯度传到浅层时变成了 0，浅层的权重根本得不到更新，模型学不到东西。
* **梯度爆炸 (Gradient Exploding)**：如果矩阵 $W$ 里面的值大多大于 1（比如 1.1），那么 $1.1^{100} \approx 13780$。梯度呈指数级放大，导致参数更新幅度极大，模型直接崩溃（Loss 变成 NaN）。

## 2. 激活函数：从根源解决非线性与梯度问题

为了让神经网络能拟合复杂的非线性曲线，必须引入非线性的激活函数。不同的激活函数对数值稳定性有着决定性的影响。



### 2.1 Sigmoid 函数
* **公式**：
  $$\sigma(x) = \frac{1}{1 + e^{-x}}$$
* **物理意义**：将任何输入映射到 $(0, 1)$ 之间，以前常用于二分类或表示概率。
* **导数公式**：
  $$\sigma'(x) = \sigma(x)(1 - \sigma(x))$$
* **致命缺陷（导致梯度消失）**：当我们画出 $\sigma'(x)$ 的图像时，会发现它的导数最大值只有 0.25（在 $x=0$ 时取得）。这意味着，每反向传播经过一层 Sigmoid，梯度最多也会被乘以 0.25。如果网络有 10 层，梯度就被缩放了 $0.25^{10} \approx 0.0000009$。因此，在深层网络中，Sigmoid 几乎被淘汰。

### 2.2 Tanh 函数 (双曲正切)
* **公式**：
  $$\text{tanh}(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}$$
* **物理意义**：将输入映射到 $(-1, 1)$ 之间。
* **优势**：它是**零中心化 (Zero-centered)** 的。相比 Sigmoid 都是正数输出，Tanh 的均值是 0，这能让下一层权重的更新方向更均匀，训练更稳定。但它依然存在梯度消失的问题（当输入很大或很小时，导数依然趋于 0）。

### 2.3 ReLU 函数 (Rectified Linear Unit)
* **公式**：
  $$
  \text{ReLU}(x) = \max(0, x)
  $$
* **导数公式**：
  $$
  \text{ReLU}'(x) =
  \begin{cases}
  1 & \text{if } x > 0 \\
  0 & \text{if } x < 0
  \end{cases}
  $$
* **为什么 ReLU 是现代深度学习的王者？**
  只要 $x > 0$，它的导数恒等于 1。这意味着无论网络有多深，在正数区间连乘时 $1 \times 1 \times 1 \dots = 1$，**完美解决了梯度消失问题**！此外，它的计算极为简单（只需要判断是否大于0），极其节省算力。

## 3. 模型初始化：Xavier 初始化的数学之美

如果不仔细初始化权重 $W$，一开始就会陷入梯度爆炸或消失。
* **不能全零初始化**：如果所有权重都是 0，那么所有神经元计算出的值都一样，反向传播时的梯度也完全一样。这就导致网络失去了“不对称性”，多个神经元退化成了一个神经元。

为了保证训练初期的稳定性，我们需要保持每一层输入和输出的方差 (Variance) 一致。这就是 **Xavier 初始化**（读作 zay-vee-er）的动机。



### Xavier 公式推导（白话解析版）

假设我们有一层线性计算：
$$h_i = \sum_{j=1}^{n_{in}} w_{ij} x_j$$
其中 $n_{in}$ 是输入特征的数量。
假设权重 $w$ 和输入 $x$ 是相互独立的，且均值为 0。

根据概率论，独立变量乘积的方差等于方差的乘积：
$$\text{Var}(h_i) = n_{in} \times \text{Var}(w_{ij}) \times \text{Var}(x_j)$$

**目标 1（前向传播稳定）**：我们希望输出的方差等于输入的方差，即 $\text{Var}(h_i) = \text{Var}(x_j)$。
因此要求：
$$n_{in} \times \text{Var}(w_{ij}) = 1 \implies \text{Var}(w_{ij}) = \frac{1}{n_{in}}$$

**目标 2（反向传播稳定）**：同理，为了保证反向传播时梯度方差不变，要求：
$$n_{out} \times \text{Var}(w_{ij}) = 1 \implies \text{Var}(w_{ij}) = \frac{1}{n_{out}}$$
其中 $n_{out}$ 是该层的输出维度。

**Xavier 的最终妥协**：由于 $n_{in}$ 和 $n_{out}$ 通常不相等，我们无法同时满足上述两个条件。Xavier 的做法是**取两者的算术平均**，得出最终的权重方差要求：
$$\text{Var}(w_{ij}) = \frac{2}{n_{in} + n_{out}}$$

### 实际应用：均匀分布的 Xavier
如果我们从均匀分布 $U[-a, a]$ 中随机采样权重，其方差为 $\frac{a^2}{3}$。
令 $\frac{a^2}{3} = \frac{2}{n_{in} + n_{out}}$，解得：
$$a = \sqrt{\frac{6}{n_{in} + n_{out}}}$$

**总结**：在 PyTorch 中使用 Xavier 均匀初始化，就是让每一层的权重随机从 $[-a, a]$ 中抽取，这从根本上保证了网络在刚开始训练时，既不会因为方差不断放大而梯度爆炸，也不会因为方差不断缩小而梯度消失。

---

通过这三块拼图：**用 ReLU 稳住连乘的梯度、用 Xavier 稳住初始的方差、时刻警惕数值稳定性的雷区**，我们就为搭建百层甚至千层的深度神经网络打下了最坚实的地基。